In [1]:
import pandas as pd
import numpy as np
from simplech import ClickHouse
from io import StringIO
from datetime import timedelta

pd.set_option('display.max_colwidth', None,"display.max_columns", None)
pd.set_option('float_format', '{:.3f}'.format)

ch = ClickHouse(host='92.255.79.45', port='8123',user='db_user', password='0987654321', db='db')

In [2]:
# Распределения по trade_cnt по разным trade_rate построить

""" Входные данные """

trade_chunk = 50 # кол-во сделок в срезе

trade_cnt = 5 # кол-во сделок с повышенным показателем подряд, которое говорит о засечки искомого паттерна

trade_rate = 3 # искомый размер увеличения показателя

trade_low_limit = 0.2 # процент нижней границы
trade_high_limit = 0.5 # процент верхней границы

time_space_for_search = 30 # время в минутах рассматриваемого периода после засечки паттерна. 1000*60*30 = пол часа в unix
time_space_24_hour = 24 # время в часах рассматриваемого периода после засечки паттерна.

clickhouse_chunk = 5_000_000 # кол-во строк в одной выгрузке из кх

ch_offset = 0

final_df = pd.DataFrame()
trade_low_limit = trade_low_limit/100
trade_high_limit = trade_low_limit/100

# узнаём дату и время, когда стопнуть цикл; делаю поправку на минуту на всякий случай
def df_from_query(query, index=None):
    query += ' FORMAT CSVWithNames'
    result = ch.select(query)
    df = pd.read_csv(StringIO(result),low_memory=False)
    if index: df.set_index(index, inplace=True)
    return df

time_to_stop = pd.to_datetime(df_from_query(f'''SELECT max(event_time) FROM binance.binancews_trade bt''').values[0], unit='ms') - timedelta(minutes=1)
is_it_time_to_stop = pd.to_datetime(df_from_query(f'''SELECT min(event_time) FROM binance.binancews_trade bt''').values[0], unit='ms')

In [3]:
trade_chunk_list = [50] # кол-во сделок в срезе
trade_cnt_list = [10,20] # кол-во сделок с повышенным показателем подряд, которое говорит о засечки искомого паттерна
trade_rate_list = [5,10] # искомый размер увеличения показателя
trade_low_limit_list = [0.2] # процент нижней границы
trade_high_limit_list = [0.5] # процент верхней границы
time_space_for_search_list = [30] # время в минутах рассматриваемого периода после засечки паттерна. 1000*60*30 = пол часа в unix
time_space_24_hour = 24 # время в часах рассматриваемого периода после засечки паттерна.
final_df = pd.DataFrame()
trade_low_limit = trade_low_limit/100
trade_high_limit = trade_low_limit/100

In [9]:
for tch in trade_chunk_list:
    print(f'{"1"*20} trade_chunk {tch} STARTS {"1"*20}')
    for tcn in trade_cnt_list:
        print(f'{"2"*20} trade_cnt {tcn} STARTS {"2"*20}')
        for tr in trade_rate_list:
            print(f'{"3"*20} trade_rate {tr} STARTS {"3"*20}')
            for tll,thl in zip(trade_low_limit_list, trade_high_limit_list):
                print(f'{"4"*20} trade_limits {tll, thl} STARTS {"4"*20}')
                for tsfs in time_space_for_search_list:
                    print(f'{"5"*20} time_space_for_search {tsfs} STARTS {"5"*20}')
                    trade_chunk = tch
                    trade_cnt = tcn
                    trade_rate = tr
                    trade_low_limit = tll
                    trade_high_limit = thl
                    time_space_for_search = tsfs
                    trade_low_limit = trade_low_limit/100
                    trade_high_limit = trade_high_limit/100
                    ch_offset = 0
                    is_it_time_to_stop = pd.to_datetime(df_from_query(f'''SELECT min(event_time) FROM binance.binancews_trade bt''').values[0], unit='ms')

                    while is_it_time_to_stop < time_to_stop:
                        print(f'OFFSET: {ch_offset} STARTS!')
                        trade_df = df_from_query(f'''
                            SELECT event_time, trade_id, price, quantity, bt."type" as trade_type, buyer_order_id, seller_order_id
                            FROM binance.binancews_trade bt
                            WHERE 1=1
                            ORDER BY 1
                            LIMIT {clickhouse_chunk}
                            OFFSET {ch_offset}
                              ''')

                        is_it_time_to_stop = pd.to_datetime(trade_df['event_time'].max(), unit='ms')

                        print(f'ch_query ok')

                        def get_mean_quantity(index):
                            """ Получаем среднее значение за предыдущие 50 тиков """
                            return trade_df['quantity'][index-trade_chunk:index+1].mean() if index > trade_chunk else trade_df['quantity'][0:index+1].mean()

                        def get_big_quantity(quantity, mean_quantity):
                            """ Вычисляем сделки, которые превышают заданные показатели """
                            trade_df['is_big'] = 0
                            trade_df.loc[(quantity > mean_quantity*trade_rate), 'is_big'] = 1
                            return trade_df['is_big']

                        def get_bignum_quantity(is_big, index, buyer_id, seller_id, trade_type):
                            """ Проставляем порядок сделок (по типу 0,0,1,2,0,0,1,2,3,4,5,0,1,2,0,1,0,0 и тп),
                            т.е. сколько узнаём сделок подряд превышают показатели, чтобы вычислить момент начала паттерна.
                            Учитываем то, что сделки подряд должен совершать один продавец или один покупатель
                            в зависимости о типа сделаки - Sell / Buy"""
                            my_list = []
                            group_num = 1
                            my_list.append(is_big[0])
                            for i, r, s, b, t in zip(index[1:len(index)+1], is_big[1:len(is_big)+1], seller_id[1:len(is_big)+1]
                                    , buyer_id[1:len(is_big)+1], trade_type[1:len(is_big)+1]):
                                if r == 0:
                                    my_list.append(0)
                                    group_num += 1
                                elif ((t == trade_type[i-1]) & (t == "Sell") & (s == seller_id[i-1])) | ((t == trade_type[i-1]) & (t == "Buy") & (b == buyer_id[i-1])):
                                    my_list.append(group_num)
                                else:
                                    group_num += 1
                                    my_list.append(group_num)
                            return my_list

                        trade_df['mean_quantity'] = np.vectorize(get_mean_quantity)(trade_df.index)
                        trade_df['is_big'] = get_big_quantity(trade_df['quantity'].values, trade_df['mean_quantity'].values)
                        trade_df['big_num_group'] = get_bignum_quantity(trade_df['is_big'].values, trade_df.index.values, trade_df['buyer_order_id'].values
                                                                        , trade_df['seller_order_id'].values, trade_df['trade_type'].values)
                        trade_df['big_num'] = trade_df.groupby('big_num_group')['is_big'].cumsum()

                        print(f'stage_1 ok')

                        def get_trade_group(big_num, index):
                            """ Разделяем итоговую выборку трейдов на тех, кто подходит под параметры искомого паттерна, и остальных """
                            my_list = []
                            for i, r, in zip(index[:len(index)-trade_cnt+1], big_num[:len(big_num)-trade_cnt+1]):
                                if (r >= trade_cnt) | (any((big_num[i+trade_cnt-x] >= trade_cnt) for x in range(1,trade_cnt))):
                                    my_list.append(1)
                                else:
                                    my_list.append(0)
                            # обработка последних строк отдельно, т.к. в цикле они вылезают за индекс
                            [my_list.append(1) if x >= trade_cnt else my_list.append(0) for x in big_num[len(big_num)-trade_cnt+1:]]
                            return my_list

                        def extreme_func(param, x):
                            """ Определяем нужно взять max или min у заданной series """
                            return x.max() if param=='Buy' else x.min() # if param=='Sell'

                        def get_final_df(df, param):
                            """ Находим все необходимые данные. Формируем конечную таблицу - df_ttl """
                            total_df = pd.DataFrame()
                            small_trade_df = trade_df[['etime_datetime', 'price']]
                            for etime in df['etime_datetime']:
                                df_ttl = pd.DataFrame()
                                df_ttl['start_etime_datetime'] = [etime]
                                df_ttl['trade_id'] = [df[df['etime_datetime'] == etime]['trade_id'].values[0]]
                                # находим данные за пол часа после начала паттерна
                                df_half_hour = small_trade_df[(small_trade_df['etime_datetime'] >= etime)
                                                              & (small_trade_df['etime_datetime'] < (etime + timedelta(minutes=time_space_for_search)))]
                                # находим данные за сутки после начала паттерна
                                df_one_day = small_trade_df[(small_trade_df['etime_datetime'] >= etime)
                                                            & (small_trade_df['etime_datetime'] < (etime + timedelta(hours=time_space_24_hour)))]
                                if param == 'Buy':
                                    # задаём верхний и нижний порог
                                    low_price = trade_df[trade_df['etime_datetime'] == etime]['price'].values[0] * (1-trade_low_limit)
                                    high_price = trade_df[trade_df['etime_datetime'] == etime]['price'].values[0] * (1+trade_high_limit)
                                    # узнаем, пересекала ли цена в заданный временной промежуток установленные ценовые пороги
                                    df_ttl['price_against'] = [True] if any(df_half_hour['price'] < low_price) else [False]
                                    df_ttl['price_for'] = [True] if any(df_half_hour['price'] > high_price) else [False]
                                elif param == 'Sell':
                                    # задаём верхний и нижний порог
                                    low_price = trade_df[trade_df['etime_datetime'] == etime]['price'].values[0] * (1-trade_high_limit)
                                    high_price = trade_df[trade_df['etime_datetime'] == etime]['price'].values[0] * (1+trade_low_limit)
                                    # узнаем, пересекала ли цена в заданный временной промежуток установленные ценовые пороги
                                    df_ttl['price_against'] = [True] if any(df_half_hour['price'] > high_price) else [False]
                                    df_ttl['price_for'] = [True] if any(df_half_hour['price'] < low_price) else [False]
                                else:
                                    print('Houston, we have a problem!')
                                price_start = df_one_day['price'].values[0]
                                price_extreme = extreme_func(param, df_one_day['price'])
                                try:
                                    if param == 'Buy':
                                        # определяем время, когда цена первый раз пересекла нижний порог
                                        first_time_against_limit = df_one_day[df_one_day['price'] < low_price]['etime_datetime'].values[0]
                                    elif param == 'Sell':
                                        # определяем время, когда цена первый раз пересекла верхний порог
                                        first_time_against_limit = df_one_day[df_one_day['price'] > high_price]['etime_datetime'].values[0]
                                    df_first_time_against_limit = df_one_day[df_one_day['etime_datetime'] < first_time_against_limit]
                                    # определяем max/min цену до первого пересечения цены нижнего/верхнего порога
                                    extreme_price_in_first_limit = extreme_func(param, df_first_time_against_limit['price'])
                                    extreme_price_time = df_one_day[df_one_day['price'] == extreme_price_in_first_limit]['etime_datetime'].values[0]
                                    start_period = df_one_day['etime_datetime'].values[0]
                                    # определяем время до max/min цены до первого пересечения цены нижнего/верхнего порога
                                    df_ttl['time_to_first_max'] = [extreme_price_time - start_period]
                                    df_ttl['first_max_percent'] = abs((1 - extreme_price_in_first_limit / price_start) * 100)
                                except Exception:
                                    # если цена за сутки не пересекала нижний/верхний порог, то проставляются пустые значения
                                    df_ttl['first_max_percent'] = [np.nan]
                                    df_ttl['time_to_first_max'] = [pd.NaT]
                                # определяем на сколько процентов ушла цена вверх/вниз максимально за сутки после начала паттерна
                                df_ttl['max_percent'] = abs((1 - price_extreme / price_start) * 100)
                                df_ttl['price_start'] = [price_start]
                                df_ttl['price_extreme'] = [price_extreme]
                                df_ttl['trade_chunk'] = [trade_chunk]
                                df_ttl['trade_cnt'] = [trade_cnt]
                                df_ttl['trade_rate'] = [trade_rate]
                                df_ttl['trade_low_limit'] = [trade_low_limit]
                                df_ttl['trade_high_limit'] = [trade_high_limit]
                                df_ttl['time_space_for_search'] = [time_space_for_search]

                                total_df = pd.concat([total_df, df_ttl], ignore_index=True)
                            return total_df

                        def final_func(param):
                            """ Разделяем таблицу на Sell и Buy.
                                Фильтруем строки, оставляем только те, которые подходят под входные данные паттерна.
                                Переводим время в datetime.
                                Оставляем только те строки, которые были за сутки до конца паттерна, иначе финальные данные будут не полны
                            """
                            param_df = trade_df[(trade_df['trade_type'] == param) & (trade_df['big_num'] > 0)].reset_index(drop=True)
                            param_df['pattern_group'] = get_trade_group(param_df['big_num'].values, param_df.index.values)
                            param_df = param_df.sort_values('event_time')
                            param_df = param_df[param_df['pattern_group'] == 1].reset_index(drop=True)
                            param_df = param_df[param_df['big_num'] == 1]
                            param_df['etime_datetime'] = pd.to_datetime(param_df['event_time'], unit='ms')
                            trade_df['etime_datetime'] = pd.to_datetime(trade_df['event_time'], unit='ms')
                            next_period_time = trade_df['etime_datetime'].max() - timedelta(hours=time_space_24_hour)
                            param_df = param_df[param_df['etime_datetime'] < next_period_time].reset_index(drop=True)
                            return get_final_df(param_df, param)

                        final_df = pd.concat([final_df, final_func('Sell')], ignore_index=True)
                        final_df = pd.concat([final_df, final_func('Buy')], ignore_index=True)
                        print(f'stage_2 ok')

                        # узнаем, сколько нужно сделать офсет строк для следующей выгрузки
                        ch_offset += len(trade_df[trade_df['etime_datetime'] < trade_df['etime_datetime'].max()-timedelta(hours=time_space_24_hour)])

                        if len(final_df) > 1_000_000:
                            break
final_df = final_df.sort_values('trade_id', ignore_index=True)
final_df.to_excel('trade.xlsx', index=False)

11111111111111111111 trade_chunk 50 STARTS 11111111111111111111
22222222222222222222 trade_cnt 10 STARTS 22222222222222222222
33333333333333333333 trade_rate 5 STARTS 33333333333333333333
44444444444444444444 trade_limits (0.2, 0.5) STARTS 44444444444444444444
55555555555555555555 time_space_for_search 30 STARTS 55555555555555555555
33333333333333333333 trade_rate 10 STARTS 33333333333333333333
44444444444444444444 trade_limits (0.2, 0.5) STARTS 44444444444444444444
55555555555555555555 time_space_for_search 30 STARTS 55555555555555555555
22222222222222222222 trade_cnt 20 STARTS 22222222222222222222
33333333333333333333 trade_rate 5 STARTS 33333333333333333333
44444444444444444444 trade_limits (0.2, 0.5) STARTS 44444444444444444444
55555555555555555555 time_space_for_search 30 STARTS 55555555555555555555
33333333333333333333 trade_rate 10 STARTS 33333333333333333333
44444444444444444444 trade_limits (0.2, 0.5) STARTS 44444444444444444444
55555555555555555555 time_space_for_search 30 ST

In [5]:
import pandas as pd
pd.read_excel('trade.xlsx')

,start_etime_datetime,trade_id,price_against,price_for,time_to_first_max,first_max_percent,max_percent,price_start,price_extreme,trade_chunk,trade_cnt,trade_rate,trade_low_limit,trade_high_limit,time_space_for_search
0,2022-01-31 23:02:00.098,1242407727,False,False,0.039,0.287,2.253,38400.020,39265.200,50,10,5,0.002,0.005,30
1,2022-02-03 04:04:00.016,1244383975,False,False,0.174,0.681,1.600,36855.950,37445.500,50,10,5,0.002,0.005,30
2,2022-02-05 03:08:00.935,1246663202,True,False,0.000,0.027,1.083,41450.800,41899.540,50,10,5,0.002,0.005,30
3,2022-02-10 11:28:00.015,1252899409,True,False,0.000,0.013,5.059,44869.760,42600.000,50,10,5,0.002,0.005,30
4,2022-02-10 21:26:06.892,1253893421,False,True,NaN,NaN,3.601,44191.230,42600.000,50,10,5,0.002,0.005,30
